In [17]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score

### Pré-processamento dos dados

In [ ]:
file_path = '../cadunico_pb_rn_full.ipynb'
data = pd.read_csv(file_path)

# removendo colunas que não se encontram em um formato adequado para um treinamento de ML
data = data.drop(columns=['dat_cadastramento_fam', 'dat_alteracao_fam', 'dat_atualizacao_familia', 'nom_estab_assist_saude_fam', 'cod_eas_fam', 'nom_centro_assist_fam', 'cod_centro_assist_fam', 'index'])

data.head()

C:\Users\luizf\AppData\Local\Temp\ipykernel_23424\4202766649.py:2: DtypeWarning: Columns (7,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


,cd_ibge,estrato,classf,id_familia,vlr_renda_media_fam,cod_local_domic_fam,cod_especie_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,...,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,cod_familia_indigena_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,peso.fam,marc_pbf
0,2405801,2,3,1573116.0,41,1.0,1.0,7.0,2.0,5.0,...,3.0,1.0,1.0,1.0,2.0,2.0,NaN,4,"4,61492813367753",1
1,2411429,1,3,2301595.0,311,1.0,1.0,5.0,1.0,2.0,...,1.0,1.0,1.0,1.0,2.0,2.0,NaN,2,"6,96709909443676",0
2,2411429,1,3,2301622.0,170,1.0,1.0,7.0,1.0,1.0,...,1.0,6.0,1.0,1.0,2.0,2.0,NaN,3,"6,96709909443676",0
3,2411429,1,3,2301623.0,50,1.0,1.0,5.0,2.0,1.0,...,1.0,1.0,1.0,1.0,2.0,2.0,NaN,4,"4,61362689750316",1
4,2411429,1,3,2301653.0,207,1.0,1.0,5.0,2.0,2.0,...,1.0,1.0,1.0,1.0,2.0,2.0,NaN,3,"6,96709909443676",0


In [3]:
# verificando linhas duplicadas devido a mesclagem de diferentes datasets
duplicados = data.duplicated(subset=['id_familia'], keep=False)

# print("Registros dublicados encontrados");
# print(data[duplicados])

data = data.drop_duplicates(subset=['id_familia'], keep='first')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1549775 entries, 0 to 1852253
Data columns (total 24 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   cd_ibge                        1549775 non-null  int64  
 1   estrato                        1549775 non-null  int64  
 2   classf                         1549775 non-null  int64  
 3   id_familia                     1549775 non-null  float64
 4   vlr_renda_media_fam            1549772 non-null  object 
 5   cod_local_domic_fam            1549110 non-null  float64
 6   cod_especie_domic_fam          1537329 non-null  float64
 7   qtd_comodos_domic_fam          1490435 non-null  float64
 8   qtd_comodos_dormitorio_fam     1489194 non-null  float64
 9   cod_material_piso_fam          1503008 non-null  float64
 10  cod_material_domic_fam         1502979 non-null  float64
 11  cod_agua_canalizada_fam        1502977 non-null  float64
 12  cod_abaste_agua_dom

In [5]:
data.describe()

,cd_ibge,estrato,classf,id_familia,cod_local_domic_fam,cod_especie_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,...,cod_banheiro_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,cod_familia_indigena_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,marc_pbf
count,1.549775e+06,1.549775e+06,1.549775e+06,1.549775e+06,1.549110e+06,1.537329e+06,1.490435e+06,1.489194e+06,1.503008e+06,1.502979e+06,...,1.502978e+06,1.403818e+06,1.502979e+06,1.502977e+06,1.489557e+06,1.549774e+06,1.545851e+06,816968.000000,1.549775e+06,1.549775e+06
mean,2.462913e+06,1.713988e+00,2.472823e+00,2.504649e+06,1.250239e+00,1.031598e+00,4.687205e+00,1.790924e+00,2.714115e+00,1.212593e+00,...,1.059475e+00,2.159214e+00,1.483216e+00,1.206869e+00,1.930749e+00,1.997469e+00,1.997024e+00,36.422128,2.996547e+00,6.638228e-01
std,5.049812e+04,4.518951e-01,7.291618e-01,1.496534e+06,4.331508e-01,1.839015e-01,1.328447e+00,6.929218e-01,1.379644e+00,7.945718e-01,...,2.365109e-01,9.862501e-01,9.279687e-01,8.090084e-01,9.781926e-01,5.024869e-02,5.446884e-02,81.333791,1.460671e+00,4.724005e-01
min,2.400109e+06,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000,1.000000e+00,0.000000e+00
25%,2.408102e+06,1.000000e+00,2.000000e+00,1.150394e+06,1.000000e+00,1.000000e+00,4.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,0.000000,2.000000e+00,0.000000e+00
50%,2.501807e+06,2.000000e+00,3.000000e+00,2.424144e+06,1.000000e+00,1.000000e+00,5.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,...,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,0.000000,3.000000e+00,1.000000e+00
75%,2.507507e+06,2.000000e+00,3.000000e+00,3.890456e+06,2.000000e+00,1.000000e+00,5.000000e+00,2.000000e+00,5.000000e+00,1.000000e+00,...,1.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00,0.000000,4.000000e+00,1.000000e+00
max,2.517407e+06,2.000000e+00,3.000000e+00,5.386861e+06,2.000000e+00,3.000000e+00,2.000000e+01,2.000000e+01,7.000000e+00,8.000000e+00,...,2.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00,306.000000,1.700000e+01,1.000000e+00


In [6]:
# colunas que precisam de tratamento de tipos
columns_to_convert = ['vlr_renda_media_fam', 'peso.fam']

for column in columns_to_convert:
    data[column] = data[column].replace(',', '.', regex=True).astype(float)

# verificando valores ausentes em cada coluna
missing_values = data.isnull().sum()

print('As colunas com valores ausentes são:\n', (missing_values))

As colunas com valores ausentes são:
 cd_ibge                               0
estrato                               0
classf                                0
id_familia                            0
vlr_renda_media_fam                   3
cod_local_domic_fam                 665
cod_especie_domic_fam             12446
qtd_comodos_domic_fam             59340
qtd_comodos_dormitorio_fam        60581
cod_material_piso_fam             46767
cod_material_domic_fam            46796
cod_agua_canalizada_fam           46798
cod_abaste_agua_domic_fam         46796
cod_banheiro_domic_fam            46797
cod_escoa_sanitario_domic_fam    145957
cod_destino_lixo_domic_fam        46796
cod_iluminacao_domic_fam          46798
cod_calcamento_domic_fam          60218
cod_familia_indigena_fam              1
ind_familia_quilombola_fam         3924
ind_parc_mds_fam                 732807
qtde_pessoas                          0
peso.fam                              0
marc_pbf                              0
dt

In [7]:
columns_to_remove = ['cod_escoa_sanitario_domic_fam', 'ind_parc_mds_fam', 'id_familia']

data = data.drop(columns=columns_to_remove, axis=1)

# removendo linhas com valores ausentes
data = data.dropna()

#missing_values = data.isnull().sum()
#print('As colunas com valores ausentes são:\n', (missing_values))
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1484779 entries, 0 to 1852253
Data columns (total 21 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   cd_ibge                     1484779 non-null  int64  
 1   estrato                     1484779 non-null  int64  
 2   classf                      1484779 non-null  int64  
 3   vlr_renda_media_fam         1484779 non-null  float64
 4   cod_local_domic_fam         1484779 non-null  float64
 5   cod_especie_domic_fam       1484779 non-null  float64
 6   qtd_comodos_domic_fam       1484779 non-null  float64
 7   qtd_comodos_dormitorio_fam  1484779 non-null  float64
 8   cod_material_piso_fam       1484779 non-null  float64
 9   cod_material_domic_fam      1484779 non-null  float64
 10  cod_agua_canalizada_fam     1484779 non-null  float64
 11  cod_abaste_agua_domic_fam   1484779 non-null  float64
 12  cod_banheiro_domic_fam      1484779 non-null  float64
 13  co

In [9]:
# identificando colunas com apenas um valor
single_value_columns = [col for col in data.columns if data[col].nunique() == 1 | 0]

print(f"Colunas com um único valor: {single_value_columns}")

data = data.drop(columns=single_value_columns)

Colunas com um único valor: ['cod_especie_domic_fam', 'cod_familia_indigena_fam']


In [10]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = data['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution))

marc_pbf
1    67.0
0    33.0
Name: proportion, dtype: float64


In [11]:
# separar X (features) e y (target)
X = data.drop('marc_pbf', axis=1)
y = data['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# aplicando SMOTE para balancear o conjunto de treino
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(y_train_balanced.value_counts())

marc_pbf
1    692773
0    692773
Name: count, dtype: int64


### Treinando o modelo RF

In [12]:
# criar e treinar o modelo
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10, class_weight='balanced')
rf_model.fit(X_train_balanced, y_train_balanced)

RandomForestClassifier(class_weight='balanced', max_depth=10, random_state=42)

In [13]:
# Fazer previsões
y_pred = rf_model.predict(X_test)

# Avaliar métricas
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Métrica ROC-AUC
y_pred_prob = rf_model.predict_proba(X_test)[:, 1]
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.86      0.78      0.82    148495
           1       0.90      0.94      0.92    296939

    accuracy                           0.89    445434
   macro avg       0.88      0.86      0.87    445434
weighted avg       0.88      0.89      0.88    445434

Matriz de Confusão:
[[116309  32186]
 [ 18633 278306]]
AUC-ROC: 0.9242251468318867


### Selecionando features com SelectKBests

In [106]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

c:\Users\luizf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 5 16] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\luizf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Características selecionadas: Index(['vlr_renda_media_fam', 'cod_material_piso_fam',
       'cod_agua_canalizada_fam', 'cod_calcamento_domic_fam', 'qtde_pessoas'],
      dtype='object')


In [107]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train_balanced)

RandomForestClassifier(random_state=42)

In [108]:
# Fazer previsões
y_pred = model.predict(X_test_selected)

# Avaliar métricas
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Métrica ROC-AUC
y_pred_prob = model.predict_proba(X_test_selected)[:, 1]
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.71      0.77    148495
           1       0.87      0.93      0.90    296939

    accuracy                           0.86    445434
   macro avg       0.85      0.82      0.83    445434
weighted avg       0.86      0.86      0.85    445434

Matriz de Confusão:
[[105838  42657]
 [ 20945 275994]]
AUC-ROC: 0.8841713188592579
